In [0]:
# Configuración del mount
container_name = "storeintelligence"

# Nombre del secret scope creado
secret_scope = "keyvault-secrets"

In [0]:
# Obtener credenciales desde Azure Key Vault
client_id = dbutils.secrets.get(scope=secret_scope, key="databricks-app-client-id")
client_secret = dbutils.secrets.get(scope=secret_scope, key="databricks-app-client-secret")
tenant_id = dbutils.secrets.get(scope=secret_scope, key="databricks-app-tenant-id")
storage_account_name = dbutils.secrets.get(scope=secret_scope, key="storage-account-name")
   
print("Credenciales obtenidas desde Key Vault exitosamente")

In [0]:
# Configuración OAuth para ADLS Gen2
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": f"{client_id}",
           "fs.azure.account.oauth2.client.secret": f"{client_secret}",
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

In [0]:
#display(configs)

In [0]:
display(dbutils.fs.mounts())

In [0]:
# Definimos la función para montar el storage account/container e incluso se podría /folder/archivo (especificarlo en mount_point)
def mount_adls(container_name):
  dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
    mount_point = f"/mnt/{container_name}",
    extra_configs = configs)

In [0]:
# Llamamos a la función enviando como parámetro el container_name
mount_adls(container_name)
print(f"Mount point {mount_point} creado exitosamente!")
print(f"Fuente: abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/")

In [0]:
display(dbutils.fs.mounts())

In [0]:
if mount_point in existing_mounts:
    dbutils.fs.unmount(f"{mount_point}")